# 🧠 BGMI Victory Classifier
Predict if a BGMI player will win or lose based on their match stats.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


## 📊 Load Dataset

In [2]:
df = pd.read_csv('BGMI_NEW_DATA.csv')
df['WinLabel'] = (df['WinPercentage'] > 0.5).astype(int)
df.head()

,Id,GroupId,MatchId,Assists,Boosts,Damage,TotalKnocks,HeadshotKills,Heals,KillPoints,...,RideDistance,RoadKills,SwimDistance,TeamKills,VehicleDestroys,WalkDistance,WeaponsAcquired,WinPoints,WinPercentage,WinLabel
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,1241,...,0.0000,0,0.00,0,0,244.80,1,1466,0.4444,0
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,0,...,0.0045,0,11.04,0,0,1434.00,5,0,0.6400,1
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,0,...,0.0000,0,0.00,0,0,161.80,2,0,0.7755,1
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,0,...,0.0000,0,0.00,0,0,202.70,3,0,0.1667,0
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,0,...,0.0000,0,0.00,0,0,49.75,2,0,0.1875,0


In [14]:
print(" Dataset Info:")
df.info()

 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               3999 non-null   object 
 1   GroupId          3999 non-null   object 
 2   MatchId          3999 non-null   object 
 3   Assists          3999 non-null   int64  
 4   Boosts           3999 non-null   int64  
 5   Damage           3999 non-null   float64
 6   TotalKnocks      3999 non-null   int64  
 7   HeadshotKills    3999 non-null   int64  
 8   Heals            3999 non-null   int64  
 9   KillPoints       3999 non-null   int64  
 10  Kills            3999 non-null   int64  
 11  KillStreaks      3999 non-null   int64  
 12  LongestKill      3999 non-null   float64
 13  MatchDuration    3999 non-null   int64  
 14  MatchType        3999 non-null   object 
 15  NumGroups        3999 non-null   int64  
 16  RankPoints       3999 non-null   int64  
 17 

In [16]:
print("\n Descriptive Statistics:")
df.describe()


 Descriptive Statistics:


,Assists,Boosts,Damage,TotalKnocks,HeadshotKills,Heals,KillPoints,Kills,KillStreaks,LongestKill,...,RideDistance,RoadKills,SwimDistance,TeamKills,VehicleDestroys,WalkDistance,WeaponsAcquired,WinPoints,WinPercentage,WinLabel
count,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,...,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000,3999.000000
mean,0.239060,1.142786,132.316162,0.658665,0.235809,1.412853,500.638660,0.949987,0.549887,22.538345,...,629.495400,0.003251,4.309238,0.025506,0.011253,1142.607741,3.640160,604.554139,0.475003,0.459365
std,0.592064,1.786047,175.226847,1.151491,0.601531,2.730306,622.997428,1.603211,0.712533,48.857249,...,1521.549936,0.056930,27.917199,0.175684,0.105494,1152.013097,2.439318,738.444995,0.309034,0.498408
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,153.900000,2.000000,0.000000,0.204100,0.000000
50%,0.000000,0.000000,81.700000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,705.500000,3.000000,0.000000,0.459200,0.000000
75%,0.000000,2.000000,185.800000,1.000000,0.000000,2.000000,1164.000000,1.000000,1.000000,20.510000,...,91.860000,0.000000,0.000000,0.000000,0.000000,1950.500000,5.000000,1495.000000,0.744700,1.000000
max,6.000000,11.000000,2325.000000,13.000000,8.000000,29.000000,1877.000000,21.000000,4.000000,417.000000,...,13510.000000,1.000000,534.500000,3.000000,1.000000,6755.000000,43.000000,1765.000000,1.000000,1.000000


## 🎯 Select Features

In [3]:
features = [
    'Assists', 'Boosts', 'Damage', 'TotalKnocks', 'HeadshotKills', 'Heals',
    'KillPoints', 'Kills', 'KillStreaks', 'LongestKill', 'MatchDuration',
    'NumGroups', 'RankPoints', 'Revives', 'RideDistance', 'RoadKills',
    'SwimDistance', 'TeamKills', 'VehicleDestroys', 'WalkDistance',
    'WeaponsAcquired', 'WinPoints'
]
X = df[features]
y = df['WinLabel']

## ✂️ Train/Test Split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 🤖 Train Random Forest Model

In [5]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## 📈 Model Evaluation

In [6]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.89      0.88       417
           1       0.88      0.86      0.87       383

    accuracy                           0.88       800
   macro avg       0.88      0.88      0.88       800
weighted avg       0.88      0.88      0.88       800



## 🧪 Predict for a New Player

In [7]:
new_data = pd.DataFrame([{
    'Assists': 2, 'Boosts': 3, 'Damage': 850, 'TotalKnocks': 1, 'HeadshotKills': 1, 'Heals': 5,
    'KillPoints': 1200, 'Kills': 6, 'KillStreaks': 1, 'LongestKill': 200, 'MatchDuration': 1800,
    'NumGroups': 20, 'RankPoints': 2000, 'Revives': 1, 'RideDistance': 0, 'RoadKills': 0,
    'SwimDistance': 0, 'TeamKills': 0, 'VehicleDestroys': 0, 'WalkDistance': 1200,
    'WeaponsAcquired': 4, 'WinPoints': 1500
}])
prediction = model.predict(new_data)
print('✅ Predicted Result:', 'Win' if prediction[0] == 1 else 'Loss')

✅ Predicted Result: Win
